In [1]:
import pandas as pd
import xarray as xr
import glob
import numpy as np
import matplotlib.pyplot as plt

In [6]:
bt = pd.read_csv('CYCLONE_BEST_TRACK_AS_FROM_IMD.csv', index_col = 0)

In [7]:
bt

,Serial Number of system during year,Basin of origin,Name,Time (UTC),Latitude (lat.),Longitude (lon.),"CI No [or ""T. No""]","Estimated Central Pressure (hPa) [or ""E.C.P""]",Maximum Sustained Surface Wind (kt),"Pressure Drop (hPa)[or ""delta P""]",Grade (text),Outermost closed isobar (hPa),Diameter/Size of outermost closed isobar(in degree),Unnamed: 14,Unnamed: 15,Unnamed: 16
Date(DD-MM-YYYY),,,,,,,,,,,,,,,,
1982-07-15,198213,ARB,NaN,300.0,20.5,69.0,1.5,NaN,25.0,3.0,D,NaN,NaN,NaN,NaN,NaN
1982-07-15,198213,ARB,NaN,1200.0,21.0,68.0,1.5,NaN,25.0,3.0,D,NaN,NaN,NaN,NaN,NaN
1982-07-16,198213,ARB,NaN,300.0,22.5,67.0,1.5,NaN,25.0,3.0,D,NaN,NaN,NaN,NaN,NaN
1982-10-01,198214,ARB,NaN,300.0,13.5,71.0,1.5,NaN,25.0,3.0,D,NaN,NaN,NaN,NaN,NaN
1982-10-01,198214,ARB,NaN,1200.0,14.0,70.0,2.0,NaN,30.0,5.0,DD,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,202306,AS,TEJ,0.0,15.9,52.1,-,990.0,60.0,18.0,SCS,NaN,NaN,NaN,NaN,NaN
2023-10-24,202306,AS,TEJ,300.0,16.0,51.8,-,996.0,50.0,12.0,SCS,NaN,NaN,NaN,NaN,NaN
2023-10-24,202306,AS,TEJ,600.0,16.1,51.5,-,1000.0,40.0,8.0,CS,NaN,NaN,NaN,NaN,NaN
